# Forecasting

In order to create better portfolios, we need to research best possible forecasting approach, for both the covariance matrix and the expected returns.

## Covariance

Baseline: histroical covariance / CovarianceShrinkage taken from PyPortfolioOpt

References:
- [Summary of classical forecasting methods](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2411493)
- [Paper doing similar work to ours, with interesing strategies to extract longer-period values from short-period predictions](http://econ.au.dk/fileadmin/site_files/filer_oekonomi/Working_Papers/CREATES/2014/rp14_42.pdf?fbclid=IwAR1PD0w4EFzq6MfJlAlzjDVCZpXmER5sVVjM7tkIVzbogZLitIaGpgGhBY8)

## Expected returns

Baseline: mean historical yearly returns

Notes:
- might be unpredictable

References: -

In [2]:
import pandas as pd
import numpy as np

from utils.data_loader import load_all

In [3]:
df = load_all()

In [4]:
df.head()

,AP,ARR,ARW,G,OP,ORR,ORW,a5.c,wig2,^aex,...,SEK,CHF,THB,TTD,TND,AED,GBP,USD,UYU,VEB
2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,415.90,549.11,354.45,401.26,275.08,520.13,230.72,1204.88,1852.9,675.44,...,0.085771,0.456726,NaN,0.115867,NaN,0.197875,NaN,0.726696,NaN,NaN
2000-01-04,404.41,533.89,357.14,401.42,275.08,520.02,229.63,1194.41,1796.6,642.25,...,NaN,0.465253,0.019568,0.115445,NaN,0.197034,1.18701,0.723608,NaN,0.001114
2000-01-05,400.04,527.38,351.19,401.59,275.08,519.22,229.22,1192.89,1777.0,632.31,...,0.086740,0.466615,0.019422,0.115510,NaN,0.197039,1.18624,0.723627,NaN,0.001114


In [5]:
df = df.loc[~df['AP'].isna()]
df.head()

,AP,ARR,ARW,G,OP,ORR,ORW,a5.c,wig2,^aex,...,SEK,CHF,THB,TTD,TND,AED,GBP,USD,UYU,VEB
2000-01-03,415.90,549.11,354.45,401.26,275.08,520.13,230.72,1204.88,1852.9,675.44,...,0.085771,0.456726,NaN,0.115867,NaN,0.197875,NaN,0.726696,NaN,NaN
2000-01-04,404.41,533.89,357.14,401.42,275.08,520.02,229.63,1194.41,1796.6,642.25,...,NaN,0.465253,0.019568,0.115445,NaN,0.197034,1.18701,0.723608,NaN,0.001114
2000-01-05,400.04,527.38,351.19,401.59,275.08,519.22,229.22,1192.89,1777.0,632.31,...,0.086740,0.466615,0.019422,0.115510,NaN,0.197039,1.18624,0.723627,NaN,0.001114
2000-01-06,410.15,522.02,347.96,401.75,275.07,519.62,228.82,NaN,1832.1,624.21,...,NaN,0.468650,0.019427,0.115662,NaN,0.197260,1.19474,0.724439,NaN,0.001115
2000-01-07,429.16,533.16,351.87,401.93,275.07,520.80,230.09,1223.61,1933.2,644.86,...,NaN,0.465233,0.019410,0.115876,NaN,0.197989,1.19596,0.727113,NaN,0.001118


In [6]:
df.shape

(4801, 200)

### Filling up missing values

In [7]:
filled = df.fillna(method='ffill')
bad_cols = [col for col in filled.columns if filled[col].loc['2001-01-01':].isna().sum() > 0]
len(bad_cols)

29

Some features have plenty of missing data. We will fill them back, but it is important to remember that we cannot leak the back-filled data into the validation dataset.

In [8]:
dont_leak_after = '2005-01-01'
cols_to_drop = [col for col in filled.columns if filled[col].loc[dont_leak_after:].isna().sum() > 0]

In [9]:
df = filled.drop(columns=cols_to_drop).fillna(method='bfill')
df.isna().sum().sum()

0

Columns that were not filled "forward" will be filled backward 

### Calculating future covariance for forecasting

Instead of using shrinkage to estimate future covariance matrix, we will use supervised learning to train a model that predicts covariance 1 year ahead.

In [49]:
from numba import jit
from tqdm import tqdm

In [50]:
returns_aggs = ['min', 'max', 'mean', 'skew', 'kurt']

def aggregated_returns(df: pd.DataFrame, aggs=returns_aggs) -> pd.DataFrame:
    returns = (df.shift(1) - df) / df
    return returns.agg(aggs)

In [77]:
%%time

dfs_to_agg_returns = [
#     df.groupby([df.index.year, df.index.month]).tail(1),
    df.groupby([df.index.year, df.index.weekofyear]).tail(1),
]

agg_returns = [aggregated_returns(aggregated_df) for aggregated_df in tqdm(dfs_to_agg_returns)]

100%|██████████| 1/1 [00:00<00:00,  2.87it/s]

CPU times: user 4.12 s, sys: 4 ms, total: 4.12 s
Wall time: 359 ms


In [84]:
%%time

dfs_to_cov = [
    df,
    df.groupby([df.index.year, df.index.month]).tail(1),
    df.groupby([df.index.year, df.index.weekofyear]).tail(1),
]

covs = [cov_df[fund_colnames].cov() for cov_df in dfs_to_cov]

CPU times: user 1.15 s, sys: 0 ns, total: 1.15 s
Wall time: 78.3 ms


In [85]:
features = np.hstack([
#     pd.concat(agg_returns, axis='columns', ignore_index=True).values.ravel(), 
    pd.concat(covs, axis='columns', ignore_index=True).values.ravel()
])

In [86]:
features.shape

(147,)

In [87]:
fund_colnames = ['AP', 'ARR', 'ARW', 'G', 'OP', 'ORR', 'ORW']

In [91]:
from typing import Tuple

In [104]:
def _single_X_y(cleaned_df, interval_start, window_size, target_size) -> Tuple[np.array, np.array]:
    timestep_present = cleaned_df.iloc[interval_start:interval_start+window_size]
    timestep_future = cleaned_df.iloc[interval_start+window_size:interval_start+window_size+target_size]
    dfs_to_cov = [
        timestep_present,
        timestep_present.groupby([timestep_present.index.year, timestep_present.index.month]).tail(1),
        timestep_present.groupby([timestep_present.index.year, timestep_present.index.weekofyear]).tail(1),
    ]
    covs = [cov_df[fund_colnames].cov() for cov_df in dfs_to_cov]
#     dfs_to_agg_returns = [
#         timestep_present.groupby([timestep_present.index.year, timestep_present.index.month]).tail(1),
#         timestep_present.groupby([timestep_present.index.year, timestep_present.index.weekofyear]).tail(1),
#     ]
#     agg_returns = [aggregated_returns(aggregated_df) for aggregated_df in tqdm(dfs_to_agg_returns)]
    features = np.hstack([
#         pd.concat(agg_returns, axis='columns', ignore_index=True).values.ravel(), 
        pd.concat(covs, axis='columns', ignore_index=True).values.ravel()
    ])
    target = timestep_future[fund_colnames].cov().values.ravel()
    return features, target

def calculate_X_y(cleaned_df: pd.DataFrame, window_size: int=750, target_size: int=250, n_features=147) -> Tuple[np.array, np.array]:
    n_samples = len(cleaned_df)-window_size-target_size
    X = np.zeros((n_samples, n_features))
    y = np.zeros((n_samples, 7**2))
    for interval_start in tqdm(range(n_samples)):
        features, target = _single_X_y(cleaned_df, interval_start, window_size, target_size)
        X[interval_start] = features
        y[interval_start] = target
    return X, y

In [105]:
%%time
X, y = calculate_X_y(df)

100%|██████████| 3801/3801 [00:52<00:00, 72.39it/s]

CPU times: user 17min 29s, sys: 316 ms, total: 17min 29s
Wall time: 52.5 s


In [106]:
X.shape

(3801, 147)

In [107]:
y.shape

(3801, 49)

## Training

In [146]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.linear_model import MultiTaskElasticNet
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor

from lightgbm import LGBMRegressor

We are not shuffling during the test set separation in order not to let the model know about macro trends that might appear in the future and are not captured within our set of input features.

In [222]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((3040, 147), (3040, 49), (761, 147), (761, 49))

In [223]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((2432, 147), (2432, 49), (608, 147), (608, 49), (761, 147), (761, 49))

In [224]:
lgbm_params = {
    'num_leaves': 31,
    'learning_rate': 0.05,
    'n_estimators': 4096,
    'silent': True,
    'n_jobs': 8,    
}

In [225]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))

In [226]:
%%time

models = {}
test_errors = {}
n_targets = y_train.shape[1]
for target_idx in range(n_targets):
    print(f"Target variable {target_idx+1} out of {n_targets}")
    train_target = y_train[:,target_idx]
    val_target = y_val[:,target_idx]
    test_target = y_test[:,target_idx]
    gbm = LGBMRegressor(**lgbm_params)
    gbm.fit(
        X_train, y_train[:,0],
        eval_set=[(X_val, y_val[:,0])],
        eval_metric='mape',
        early_stopping_rounds=64,
        verbose=512
    )
    y_pred = gbm.predict(X_test)
    test_err = mean_absolute_percentage_error(y_test[:,target_idx], y_pred)
    print(f"Test set error: {test_err:.4f}")
    models[target_idx] = gbm
    test_errors[target_idx] = test_err

Target variable 1 out of 49
Training until validation scores don't improve for 64 rounds.
Early stopping, best iteration is:
[136]	valid_0's mape: 2.6914	valid_0's l2: 4.92006e+06
Test set error: 2.2516
Target variable 2 out of 49
Training until validation scores don't improve for 64 rounds.
Early stopping, best iteration is:
[136]	valid_0's mape: 2.6914	valid_0's l2: 4.92006e+06
Test set error: 21.3792
Target variable 3 out of 49
Training until validation scores don't improve for 64 rounds.
Early stopping, best iteration is:
[136]	valid_0's mape: 2.6914	valid_0's l2: 4.92006e+06
Test set error: 4.2642
Target variable 4 out of 49
Training until validation scores don't improve for 64 rounds.
Early stopping, best iteration is:
[136]	valid_0's mape: 2.6914	valid_0's l2: 4.92006e+06
Test set error: 190.2276
Target variable 5 out of 49
Training until validation scores don't improve for 64 rounds.
Early stopping, best iteration is:
[136]	valid_0's mape: 2.6914	valid_0's l2: 4.92006e+06
Test 

As we can see, the error rates are pretty bad and forecasting covariance using this set of data is not the greatest idea.

### Adding Shrunk covariance matrix as feature to the model
Instead of forecasting the correct future covariance for the portfolio, we will train the model as a 2nd-stage estimator, stacked on predictions from existing widely used covariance shrinkage techniques.

To do this, we will train a fully-connected neural net with a loss that corresponds to future portfolio performance.